# DrivenData Water Table: An Introduction

In the previous notebook, you learned some basic statistical visualization techniques. Now, we'll go over basic data cleaning techniques and how to get started on making machine learning models.

Let's start by reading in the data.

In [1]:
import pandas as pd 
import sklearn as sk
import numpy as np
import seaborn as sns

df = pd.read_csv('../data-visualization/train_features.csv')
df_labels = pd.read_csv('../data-visualization/train_labels.csv')
df_test = pd.read_csv('../data-visualization/test_features.csv')

display(df.head(5))
display(df_labels.head(5))

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


## 1. Data Cleaning 

And viewing what percentage of each column is missing 

In [2]:
df.isna().sum().sort_values(ascending=False) / df.shape[0] * 100

scheme_name              47.417508
scheme_management         6.526936
installer                 6.153199
funder                    6.119529
public_meeting            5.612795
permit                    5.144781
subvillage                0.624579
ward                      0.000000
lga                       0.000000
district_code             0.000000
region_code               0.000000
region                    0.000000
waterpoint_type_group     0.000000
population                0.000000
num_private               0.000000
wpt_name                  0.000000
latitude                  0.000000
longitude                 0.000000
gps_height                0.000000
date_recorded             0.000000
amount_tsh                0.000000
basin                     0.000000
recorded_by               0.000000
waterpoint_type           0.000000
water_quality             0.000000
source_class              0.000000
source_type               0.000000
source                    0.000000
quantity_group      

As we can see, almost 50% of scheme name values are missing. Since this is a categorical variable, we'll fill the missing values with the string 'missing', and then create one-hot encodings.

For the other variables, will drop the rows with any NaN (missing) values. Although this technique isn't recommended for serious modeling, we are just doing some quick and dirty initial tests, so missing 6% of the rows on a fairly large dataset should be okay. 

In [3]:
df['scheme_name'] = df['scheme_name'].fillna('Missing')
df = df.dropna()
df.isna().sum()

id                       0
amount_tsh               0
date_recorded            0
funder                   0
gps_height               0
installer                0
longitude                0
latitude                 0
wpt_name                 0
num_private              0
basin                    0
subvillage               0
region                   0
region_code              0
district_code            0
lga                      0
ward                     0
population               0
public_meeting           0
recorded_by              0
scheme_management        0
scheme_name              0
permit                   0
construction_year        0
extraction_type          0
extraction_type_group    0
extraction_type_class    0
management               0
management_group         0
payment                  0
payment_type             0
water_quality            0
quality_group            0
quantity                 0
quantity_group           0
source                   0
source_type              0
s

Ultimately, we need our data to be numerical in order to feed it into some statistical models. To do this, we'll encode each categorical variable into an indicator variable. For example, if a water pump was funded by `x`, the column `funded_x` will mark a 1. 

Since our dataset is mostly categorical variables, this encoding might take a while to run. If it doesn't complete in a reasonable time, try it on google Colab using the link in the GitHub repo. 

In [4]:
df = pd.get_dummies(df)
df.isna().sum().sum() # Take the sum over the Series returned by df.isna().sum()

0

In [5]:
df.head(5)

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year,...,waterpoint_type_dam,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other,waterpoint_type_group_cattle trough,waterpoint_type_group_communal standpipe,waterpoint_type_group_dam,waterpoint_type_group_hand pump,waterpoint_type_group_improved spring,waterpoint_type_group_other
0,69572,6000.0,1390,34.938093,-9.856322,0,11,5,109,1999,...,0,0,0,0,0,1,0,0,0,0
2,34310,25.0,686,37.460664,-3.821329,0,21,4,250,2009,...,0,0,0,0,0,1,0,0,0,0
3,67743,0.0,263,38.486161,-11.155298,0,90,63,58,1986,...,0,0,0,0,0,1,0,0,0,0
5,9944,20.0,0,39.172796,-4.765587,0,4,8,1,2009,...,0,0,0,0,0,1,0,0,0,0
6,19816,0.0,0,33.362410,-3.766365,0,17,3,0,0,...,0,1,0,0,0,0,0,1,0,0


Now, we can start doing some basic modeling. First, let's review what we're trying to predict in this competition.

In [6]:
df_labels['status_group'].value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

So we have a categorical variable with three options (or classes). Let's create a new category mapping each option to a numeric code to be used in our classifier. We'll do this using a bit of Pandas magic that we'll explain during our meeting in more detail (although, like most Pandas, is quite intuitive!).

In [7]:
df_labels['categorical_status_group'] = df_labels['status_group'].replace(
    to_replace=['functional', 'non functional', 'functional needs repair'],
    value=[0,1,2])

df_labels = df_labels.drop('status_group', axis=1)
df_labels

,id,categorical_status_group
0,69572,0
1,8776,0
2,34310,0
3,67743,1
4,19728,0
...,...,...
59395,60739,0
59396,27263,0
59397,37057,0
59398,31282,0


Now we have created three classes, each corresponding to a unique value in our label set. Let's now join our two DataFrames on their `id` and then get to the good stuff!

In [8]:
df = pd.merge(df, df_labels, on='id')

Let's drop `id`, as it isn't a relevant predictor but rather a feature of how our data is structured. As you're becoming more familiar with Pandas, notice that we're dropping along `axis=1`, which refers to columns. In this case, the `.drop()` method automatically assumes rows, so this specification is necessary. Regardless, it's always good to be explicit!

In [9]:
df = df.drop('id', axis=1)

In [10]:
df

,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year,date_recorded_2002-10-14,...,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other,waterpoint_type_group_cattle trough,waterpoint_type_group_communal standpipe,waterpoint_type_group_dam,waterpoint_type_group_hand pump,waterpoint_type_group_improved spring,waterpoint_type_group_other,categorical_status_group
0,6000.0,1390,34.938093,-9.856322,0,11,5,109,1999,0,...,0,0,0,0,1,0,0,0,0,0
1,25.0,686,37.460664,-3.821329,0,21,4,250,2009,0,...,0,0,0,0,1,0,0,0,0,0
2,0.0,263,38.486161,-11.155298,0,90,63,58,1986,0,...,0,0,0,0,1,0,0,0,0,1
3,20.0,0,39.172796,-4.765587,0,4,8,1,2009,0,...,0,0,0,0,1,0,0,0,0,0
4,0.0,0,33.362410,-3.766365,0,17,3,0,0,0,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48283,500.0,351,37.634053,-6.124830,0,5,6,89,2007,0,...,0,0,0,0,1,0,0,0,0,1
48284,10.0,1210,37.169807,-3.253847,0,3,5,125,1999,0,...,0,0,0,0,1,0,0,0,0,0
48285,4700.0,1212,35.249991,-9.070629,0,11,4,56,1996,0,...,0,0,0,0,1,0,0,0,0,0
48286,0.0,0,35.861315,-6.378573,0,1,4,0,0,0,...,1,0,0,0,0,0,1,0,0,0


## 2. Modeling 

In this section, we'll build some basic classifiers using the `sklearn` API, and discuss some fundamental methods for assessing our models. 

In [11]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

knn = KNeighborsClassifier()
logistic = LogisticRegression()

After training our models, we'd like to get a measure of how well they perform. However, it isn't just good enough to measure their score on the training data. What we really care about is how the model *generalizes*, that is, how it performs on unseen data. 

So how do we get unseen data? Well, let's just split our dataset into two subsets, training on one subset, and evaluating on the other. 

In [12]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('categorical_status_group', axis=1), df['categorical_status_group'])

In [13]:
X_train.shape, X_test.shape

((36216, 55649), (12072, 55649))

## More features than observations

When we encoded the categorical variables, we ended up with more features than data points, by nearly 20k. This is an issue, because 